In [1]:
# %%
# Imports
import os
import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, CrossEncoder
from ollama import Client

In [2]:
# %%
# Load environment variables
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [3]:
# %%
# Initialize clients
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("funding-search-bge")
embed_model = SentenceTransformer("BAAI/bge-small-en")
rerank_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
ollama_client = Client(host="http://localhost:11434")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [4]:
# %%
# Input query
query = "We are an AI company focused on AI for robotics. We are focusing on research right now."
query_embedding = embed_model.encode(query).tolist()

In [5]:
# %%
# Get semantic matches from Pinecone
semantic_matches = index.query(
    vector=query_embedding,
    top_k=8,
    include_metadata=True,
    namespace="open-source-v1"
)

In [ ]:
for match in semantic_matches["matches"]:
    print(match["metadata"]["name"])

In [6]:
# %%
# Rerank using CrossEncoder
rerank_inputs = [
    [query, match["metadata"].get("description", "")] for match in semantic_matches["matches"]
]
scores = rerank_model.predict(rerank_inputs)
reranked = [x for _, x in sorted(zip(scores, semantic_matches["matches"]), reverse=True)]
top_matches = reranked[:5]

In [7]:
# %%
# Format structured funding blocks with citations
def generate_funding_blocks(matches, user_query: str) -> str:
    formatted = []
    for idx, match in enumerate(matches, start=1):
        meta = match["metadata"]
        name = meta.get("name", f"Program {idx}")
        block = f"""**{idx}. {name}**\n"""
        fields = {
            "Description": meta.get("description"),
            "Domain": meta.get("domain"),
            "Eligibility": meta.get("eligibility"),
            "Amount": meta.get("amount"),
            "Deadline": meta.get("deadline"),
            "Procedure": meta.get("procedure"),
            "Contact": meta.get("contact"),
            "URL": meta.get("url")
        }
        for key, val in fields.items():
            if val and "not found" not in str(val).lower():
                block += f"- **{key}**: {val}\n"
        block += "\n"
        formatted.append(block)
    return "\n".join(formatted).strip()

In [8]:
# %%
semantic_output = generate_funding_blocks(top_matches, query)
print("🔍 Top Results:\n", semantic_output[:1000], "...\n")

🔍 Top Results:
 **1. AIRISE Open Call**
- **Description**: AIRISE supports SMEs and mid-caps in the implementation and scaling of AI solutions in manufacturing. The Open Calls are aimed at companies that are either conducting their first AI experiments or want to bring existing applications to a higher level of maturity (TRL). Funding is available for projects in areas such as design & engineering, process monitoring, manufacturing operations, supply chain, cyber security and training Further information can be found here
- **Domain**: AI in production
- **Eligibility**: SMEs and mid-caps
- **Amount**: Up to 60,000 euros
- **Deadline**: July 16, 2025
- **URL**: https://airise.eu/calls


**2. Funding of collaborative research projects in the fields of "software and algorithms with a focus on artificial intelligence and machine learning", "research data management" and "federated digital infrastructures" for the exploration of the universe and matter (ErUM)**
- **Description**: Directive

In [ ]:
# %%
# Load the advanced prompt template
with open("llm_prompt_llama_advanced.txt", "r") as f:
    prompt_template = f.read()

llm_prompt = prompt_template.replace("{{QUERY}}", query).replace("{{RESULTS}}", semantic_output)

In [ ]:
# %%
# Generate recommendation using LLaMA 3.2
response = ollama_client.generate(
    model="llama3.2",
    prompt=llm_prompt,
    stream=False
)

In [ ]:
# %%
# Print final output
print("🧾 Final Recommendation:\n")
print(response["response"])
